# spark sharing


In [177]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.classification import LogisticRegressionSummary

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import lit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorSlicer
from pyspark.sql.functions import regexp_replace
# from pyspark.sql.functions import when
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vector

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer

import numpy as np
import pandas as pd

In [ ]:
1.basis

2.LR

3.GBDT

In [ ]:
Estimator:
An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. 
E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.

Transformer:
A Transformer is an algorithm which can transform one DataFrame into another DataFrame. 
E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.

## 1、basis

### 1.1 SparkSession是DataFrame和SQL功能的主入口

In [68]:
# spark = SparkSession.builder.master("spark://10.0.120.106:7077").appName("spark_app").config("", "").getOrCreate()

# spark = SparkSession.builder.master("local").appName("spark_app").config("", "").getOrCreate()

# main entry of programming Spark with DataFrame. 'enableHiveSupport()' enables spark to utilize SQL or HQL
# A SparkSession can be used create DataFrame, register DataFrame as tables, execute SQL over tables, cache tables, and read parquet files.
spark = SparkSession.builder.master("local").appName("spark_app").config("", "").enableHiveSupport().getOrCreate()


In [69]:
spark

### 1.2 读取数据文件

In [70]:
# 虽然是txt格式，但是按照csv来读取可以设置seperator
read_data = spark.read.csv('C:\spark\data\my_data\iris_data.txt', sep=',')
read_data.head(5)

[Row(_c0='5.1', _c1='3.5', _c2='1.4', _c3='0.2', _c4='Iris-setosa'),
 Row(_c0='4.9', _c1='3.0', _c2='1.4', _c3='0.2', _c4='Iris-setosa'),
 Row(_c0='4.7', _c1='3.2', _c2='1.3', _c3='0.2', _c4='Iris-setosa'),
 Row(_c0='4.6', _c1='3.1', _c2='1.5', _c3='0.2', _c4='Iris-setosa'),
 Row(_c0='5.0', _c1='3.6', _c2='1.4', _c3='0.2', _c4='Iris-setosa')]

In [ ]:
## 读写hive
# df = spark.sql('select * from xxx')

### 1.3 拆分数据集

In [71]:
train_data, test_data = read_data.randomSplit([0.7, 0.3], 1)

In [72]:
train_data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]

In [73]:
test_data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]

### 1.4 操作dataframe

#### A distributed collection of data grouped into named columns. A DataFrame is equivalent to a relational table in Spark SQL

In [74]:
# show the structure of dataframe - [train_data]
train_data.show(50)

+---+---+---+---+---------------+
|_c0|_c1|_c2|_c3|            _c4|
+---+---+---+---+---------------+
|4.3|3.0|1.1|0.1|    Iris-setosa|
|4.4|2.9|1.4|0.2|    Iris-setosa|
|4.4|3.0|1.3|0.2|    Iris-setosa|
|4.4|3.2|1.3|0.2|    Iris-setosa|
|4.6|3.1|1.5|0.2|    Iris-setosa|
|4.6|3.2|1.4|0.2|    Iris-setosa|
|4.6|3.4|1.4|0.3|    Iris-setosa|
|4.6|3.6|1.0|0.2|    Iris-setosa|
|4.7|3.2|1.3|0.2|    Iris-setosa|
|4.8|3.0|1.4|0.1|    Iris-setosa|
|4.8|3.0|1.4|0.3|    Iris-setosa|
|4.8|3.1|1.6|0.2|    Iris-setosa|
|4.8|3.4|1.6|0.2|    Iris-setosa|
|4.8|3.4|1.9|0.2|    Iris-setosa|
|4.9|2.5|4.5|1.7| Iris-virginica|
|4.9|3.1|1.5|0.1|    Iris-setosa|
|4.9|3.1|1.5|0.1|    Iris-setosa|
|5.0|2.0|3.5|1.0|Iris-versicolor|
|5.0|2.3|3.3|1.0|Iris-versicolor|
|5.0|3.2|1.2|0.2|    Iris-setosa|
|5.0|3.3|1.4|0.2|    Iris-setosa|
|5.0|3.4|1.5|0.2|    Iris-setosa|
|5.0|3.5|1.3|0.3|    Iris-setosa|
|5.0|3.5|1.6|0.6|    Iris-setosa|
|5.0|3.6|1.4|0.2|    Iris-setosa|
|5.1|3.3|1.7|0.5|    Iris-setosa|
|5.1|3.4|1.5|0

In [75]:
train_data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]

In [76]:
# get all columns' names
train_data.columns

['_c0', '_c1', '_c2', '_c3', '_c4']

In [77]:
# get the num of rows
train_data.count()

106

In [78]:
# filter
train_data.filter(train_data._c3=='0.1').show()

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|4.3|3.0|1.1|0.1|Iris-setosa|
|4.8|3.0|1.4|0.1|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|5.2|4.1|1.5|0.1|Iris-setosa|
+---+---+---+---+-----------+



In [79]:
# contains
train_data.filter(train_data._c4.contains('virginica')).show(5)

+---+---+---+---+--------------+
|_c0|_c1|_c2|_c3|           _c4|
+---+---+---+---+--------------+
|4.9|2.5|4.5|1.7|Iris-virginica|
|5.6|2.8|4.9|2.0|Iris-virginica|
|5.7|2.5|5.0|2.0|Iris-virginica|
|5.8|2.7|5.1|1.9|Iris-virginica|
|5.8|2.7|5.1|1.9|Iris-virginica|
+---+---+---+---+--------------+
only showing top 5 rows



In [80]:
# The available aggregate functions are avg, max, min, sum, count.
train_data.agg({'_c3': 'max'}).show()

+--------+
|max(_c3)|
+--------+
|     2.5|
+--------+



In [81]:
train_data.agg({'_c3': 'avg'}).show()

+-----------------+
|         avg(_c3)|
+-----------------+
|1.161320754716981|
+-----------------+



In [82]:
train_data.agg({'_c3': 'sum'}).show()

+------------------+
|          sum(_c3)|
+------------------+
|123.09999999999998|
+------------------+



In [83]:
# Creates or replaces a local temporary view
train_data.createOrReplaceTempView('train_table')

In [84]:
# query like sql
train_1 = spark.sql("select * from train_table where _c4=='Iris-setosa'")
train_1.show()

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|4.3|3.0|1.1|0.1|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.4|3.0|1.3|0.2|Iris-setosa|
|4.4|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|4.6|3.2|1.4|0.2|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|4.6|3.6|1.0|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.8|3.0|1.4|0.1|Iris-setosa|
|4.8|3.0|1.4|0.3|Iris-setosa|
|4.8|3.1|1.6|0.2|Iris-setosa|
|4.8|3.4|1.6|0.2|Iris-setosa|
|4.8|3.4|1.9|0.2|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|5.0|3.2|1.2|0.2|Iris-setosa|
|5.0|3.3|1.4|0.2|Iris-setosa|
|5.0|3.4|1.5|0.2|Iris-setosa|
|5.0|3.5|1.3|0.3|Iris-setosa|
+---+---+---+---+-----------+
only showing top 20 rows



In [85]:
# describe the dataframe by providing basic statistics for numeric and string columns
train_data.describe().show()

+-------+------------------+-------------------+------------------+------------------+--------------+
|summary|               _c0|                _c1|               _c2|               _c3|           _c4|
+-------+------------------+-------------------+------------------+------------------+--------------+
|  count|               106|                106|               106|               106|           106|
|   mean| 5.770754716981132|  3.071698113207549|3.6443396226415086| 1.161320754716981|          null|
| stddev|0.7997579373228328|0.44588162362012673|1.7494532581939912|0.7643946568198022|          null|
|    min|               4.3|                2.0|               1.0|               0.1|   Iris-setosa|
|    max|               7.9|                4.4|               6.9|               2.5|Iris-virginica|
+-------+------------------+-------------------+------------------+------------------+--------------+



In [86]:
# add some quartiles statistics infomation
train_data.summary().show()

+-------+------------------+-------------------+------------------+------------------+--------------+
|summary|               _c0|                _c1|               _c2|               _c3|           _c4|
+-------+------------------+-------------------+------------------+------------------+--------------+
|  count|               106|                106|               106|               106|           106|
|   mean| 5.770754716981132|  3.071698113207549|3.6443396226415086| 1.161320754716981|          null|
| stddev|0.7997579373228328|0.44588162362012673|1.7494532581939912|0.7643946568198022|          null|
|    min|               4.3|                2.0|               1.0|               0.1|   Iris-setosa|
|    25%|               5.1|                2.8|               1.5|               0.3|          null|
|    50%|               5.7|                3.0|               4.1|               1.3|          null|
|    75%|               6.3|                3.4|               5.1|               

In [87]:
# summary for specific column and statistics
train_data.select('_c0').summary('mean').show()

+-------+-----------------+
|summary|              _c0|
+-------+-----------------+
|   mean|5.770754716981132|
+-------+-----------------+



In [88]:
train_data.select('_c0')

DataFrame[_c0: string]

In [89]:
train_data.select('_c0').show(10)

+---+
|_c0|
+---+
|4.3|
|4.4|
|4.4|
|4.4|
|4.6|
|4.6|
|4.6|
|4.6|
|4.7|
|4.8|
+---+
only showing top 10 rows



In [90]:
train_data.select('_c0').collect()

[Row(_c0='4.3'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.6'),
 Row(_c0='4.6'),
 Row(_c0='4.6'),
 Row(_c0='4.6'),
 Row(_c0='4.7'),
 Row(_c0='4.8'),
 Row(_c0='4.8'),
 Row(_c0='4.8'),
 Row(_c0='4.8'),
 Row(_c0='4.8'),
 Row(_c0='4.9'),
 Row(_c0='4.9'),
 Row(_c0='4.9'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.1'),
 Row(_c0='5.1'),
 Row(_c0='5.1'),
 Row(_c0='5.1'),
 Row(_c0='5.2'),
 Row(_c0='5.2'),
 Row(_c0='5.3'),
 Row(_c0='5.4'),
 Row(_c0='5.4'),
 Row(_c0='5.4'),
 Row(_c0='5.4'),
 Row(_c0='5.4'),
 Row(_c0='5.5'),
 Row(_c0='5.5'),
 Row(_c0='5.5'),
 Row(_c0='5.5'),
 Row(_c0='5.5'),
 Row(_c0='5.6'),
 Row(_c0='5.6'),
 Row(_c0='5.6'),
 Row(_c0='5.6'),
 Row(_c0='5.6'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.8'),
 Row(_c0='5.8'),
 Row(_c0='5.8'),
 Row(_c0='5.8'

In [91]:
train_data.select('_c0').head(5)

[Row(_c0='4.3'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.6')]

In [92]:
type(train_data.select('_c0').head(5))

list

In [93]:
train_data.select('_c0').take(5)

[Row(_c0='4.3'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.6')]

In [94]:
train_data.select('_c0').take(5)[0]

Row(_c0='4.3')

In [95]:
type(train_data.select('_c0').take(5)[0])

pyspark.sql.types.Row

In [96]:
train_data.select('_c0').take(5)[0][0]

'4.3'

In [97]:
type(train_data.select('_c0').take(5)[0][0])

str

### 1.5 操作column

In [98]:
# pick a column in a dataframe
train_data._c0

Column<b'_c0'>

In [99]:
train_data['_c0']

Column<b'_c0'>

In [100]:
# show the result - 'column' object is not callable
train_data['_c0'].collect()

TypeError: 'Column' object is not callable

In [101]:
# do calculations on dataframe through column
operate_data = train_data.select(['_c0', '_c1'])
operate_data
operate_data.show(5)

+---+---+
|_c0|_c1|
+---+---+
|4.3|3.0|
|4.4|2.9|
|4.4|3.0|
|4.4|3.2|
|4.6|3.1|
+---+---+
only showing top 5 rows



In [102]:
# sum
add_col = operate_data._c0 + operate_data['_c1']
add_col

Column<b'(_c0 + _c1)'>

In [103]:
add_operate_data = operate_data.withColumn('sum', add_col)

In [104]:
add_operate_data.show(5)

+---+---+------------------+
|_c0|_c1|               sum|
+---+---+------------------+
|4.3|3.0|               7.3|
|4.4|2.9| 7.300000000000001|
|4.4|3.0|               7.4|
|4.4|3.2|7.6000000000000005|
|4.6|3.1| 7.699999999999999|
+---+---+------------------+
only showing top 5 rows



## 1、LR

In [105]:
train_data.show(10)

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|4.3|3.0|1.1|0.1|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.4|3.0|1.3|0.2|Iris-setosa|
|4.4|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|4.6|3.2|1.4|0.2|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|4.6|3.6|1.0|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.8|3.0|1.4|0.1|Iris-setosa|
+---+---+---+---+-----------+
only showing top 10 rows



In [109]:
stringIndexer = StringIndexer(inputCol='_c4', outputCol='indexedLabel')
stringIndexer_model = stringIndexer.fit(train_data)
train_data_2 = stringIndexer_model.transform(train_data)
train_data_2.show(20)

+---+---+---+---+---------------+------------+
|_c0|_c1|_c2|_c3|            _c4|indexedLabel|
+---+---+---+---+---------------+------------+
|4.3|3.0|1.1|0.1|    Iris-setosa|         0.0|
|4.4|2.9|1.4|0.2|    Iris-setosa|         0.0|
|4.4|3.0|1.3|0.2|    Iris-setosa|         0.0|
|4.4|3.2|1.3|0.2|    Iris-setosa|         0.0|
|4.6|3.1|1.5|0.2|    Iris-setosa|         0.0|
|4.6|3.2|1.4|0.2|    Iris-setosa|         0.0|
|4.6|3.4|1.4|0.3|    Iris-setosa|         0.0|
|4.6|3.6|1.0|0.2|    Iris-setosa|         0.0|
|4.7|3.2|1.3|0.2|    Iris-setosa|         0.0|
|4.8|3.0|1.4|0.1|    Iris-setosa|         0.0|
|4.8|3.0|1.4|0.3|    Iris-setosa|         0.0|
|4.8|3.1|1.6|0.2|    Iris-setosa|         0.0|
|4.8|3.4|1.6|0.2|    Iris-setosa|         0.0|
|4.8|3.4|1.9|0.2|    Iris-setosa|         0.0|
|4.9|2.5|4.5|1.7| Iris-virginica|         2.0|
|4.9|3.1|1.5|0.1|    Iris-setosa|         0.0|
|4.9|3.1|1.5|0.1|    Iris-setosa|         0.0|
|5.0|2.0|3.5|1.0|Iris-versicolor|         1.0|
|5.0|2.3|3.3|

In [111]:
# check the maps relation
sorted( set([(i[0], i[1]) for i in train_data_2.select(['_c4', 'indexedLabel']).collect()]), key=lambda x:x[0] )

[('Iris-setosa', 0.0), ('Iris-versicolor', 1.0), ('Iris-virginica', 2.0)]

In [115]:
# transform string to double
train_data = train_data.withColumn('_c0', train_data['_c0'].cast('double'))\
       .withColumn('_c1', train_data['_c1'].cast('double'))\
       .withColumn('_c2', train_data['_c2'].cast('double'))\
       .withColumn('_c3', train_data['_c3'].cast('double'))

In [116]:
##### pipeline #####
# assemble features to vector and indicate the label
input_col = ['_c0', '_c1', '_c2', '_c3']
vecAssembler = VectorAssembler(inputCols=input_col, outputCol="features")
# new_train_data = vecAssembler.transform(train_data)
stringIndexer = StringIndexer(inputCol="_c4", outputCol="label")
pipeline = Pipeline(stages=[vecAssembler, stringIndexer])
pipelineFit = pipeline.fit(train_data)
new_train_data = pipelineFit.transform(train_data)

In [118]:
new_train_data.show(20)

+---+---+---+---+---------------+-----------------+-----+
|_c0|_c1|_c2|_c3|            _c4|         features|label|
+---+---+---+---+---------------+-----------------+-----+
|4.3|3.0|1.1|0.1|    Iris-setosa|[4.3,3.0,1.1,0.1]|  0.0|
|4.4|2.9|1.4|0.2|    Iris-setosa|[4.4,2.9,1.4,0.2]|  0.0|
|4.4|3.0|1.3|0.2|    Iris-setosa|[4.4,3.0,1.3,0.2]|  0.0|
|4.4|3.2|1.3|0.2|    Iris-setosa|[4.4,3.2,1.3,0.2]|  0.0|
|4.6|3.1|1.5|0.2|    Iris-setosa|[4.6,3.1,1.5,0.2]|  0.0|
|4.6|3.2|1.4|0.2|    Iris-setosa|[4.6,3.2,1.4,0.2]|  0.0|
|4.6|3.4|1.4|0.3|    Iris-setosa|[4.6,3.4,1.4,0.3]|  0.0|
|4.6|3.6|1.0|0.2|    Iris-setosa|[4.6,3.6,1.0,0.2]|  0.0|
|4.7|3.2|1.3|0.2|    Iris-setosa|[4.7,3.2,1.3,0.2]|  0.0|
|4.8|3.0|1.4|0.1|    Iris-setosa|[4.8,3.0,1.4,0.1]|  0.0|
|4.8|3.0|1.4|0.3|    Iris-setosa|[4.8,3.0,1.4,0.3]|  0.0|
|4.8|3.1|1.6|0.2|    Iris-setosa|[4.8,3.1,1.6,0.2]|  0.0|
|4.8|3.4|1.6|0.2|    Iris-setosa|[4.8,3.4,1.6,0.2]|  0.0|
|4.8|3.4|1.9|0.2|    Iris-setosa|[4.8,3.4,1.9,0.2]|  0.0|
|4.9|2.5|4.5|1

In [120]:
new_train_data.select('features')

DataFrame[features: vector]

In [142]:
new_train_data.select('features').collect()[0][0].toArray()

array([4.3, 3. , 1.1, 0.1])

In [147]:
# transform test_data using pipeline
test_data = test_data.withColumn('_c0', test_data['_c0'].cast('double'))\
       .withColumn('_c1', test_data['_c1'].cast('double'))\
       .withColumn('_c2', test_data['_c2'].cast('double'))\
       .withColumn('_c3', test_data['_c3'].cast('double'))

new_test_data = pipelineFit.transform(test_data)

In [149]:
new_test_data.show(5)

+---+---+---+---+---------------+-----------------+-----+
|_c0|_c1|_c2|_c3|            _c4|         features|label|
+---+---+---+---+---------------+-----------------+-----+
|4.5|2.3|1.3|0.3|    Iris-setosa|[4.5,2.3,1.3,0.3]|  0.0|
|4.7|3.2|1.6|0.2|    Iris-setosa|[4.7,3.2,1.6,0.2]|  0.0|
|4.9|2.4|3.3|1.0|Iris-versicolor|[4.9,2.4,3.3,1.0]|  1.0|
|4.9|3.0|1.4|0.2|    Iris-setosa|[4.9,3.0,1.4,0.2]|  0.0|
|4.9|3.1|1.5|0.1|    Iris-setosa|[4.9,3.1,1.5,0.1]|  0.0|
+---+---+---+---+---------------+-----------------+-----+
only showing top 5 rows



#### train model

In [ ]:
"""
class pyspark.ml.classification.LogisticRegression(featuresCol='features', labelCol='label', predictionCol='prediction', 
maxIter=100, regParam=0.0, elasticNetParam=0.0, tol=1e-06, fitIntercept=True, threshold=0.5, thresholds=None,probabilityCol='probability', 
rawPredictionCol='rawPrediction', standardization=True, weightCol=None, aggregationDepth=2, family='auto', 
lowerBoundsOnCoefficients=None, upperBoundsOnCoefficients=None, lowerBoundsOnIntercepts=None, upperBoundsOnIntercepts=None)
"""

In [180]:
# default -> pick 'features' and 'label' columns and input them into the LR model 
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.7)
lr_model = lr.fit(new_train_data)
lr_model

LogisticRegression_48f48c85243b437dced4

In [181]:
model_summary = lr_model.summary

In [182]:
# display the objective per iteration
objectiveHistory = model_summary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

objectiveHistory:
1.0963514880151817
1.0893542842612507
1.054540991144702
1.0124314678821982
1.011233434646947
1.0074285594594459
1.0063296684786371
1.0055798406732783
1.0049471652881403
1.00441485771545
1.0039481425760803


In [183]:
# model_summary.roc.show()
print("areaUnderROC: " + str(model_summary.areaUnderROC))

AttributeError: 'LogisticRegressionTrainingSummary' object has no attribute 'areaUnderROC'

In [171]:
print("True positive rate by label:")
for i, rate in enumerate(model_summary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

True positive rate by label:
label 0: 1.0
label 1: 0.972972972972973
label 2: 0.96875


In [151]:
prediction = lr_model.transform(new_test_data)

In [154]:
prediction.show(10)

+---+---+---+---+---------------+-----------------+-----+--------------------+--------------------+----------+
|_c0|_c1|_c2|_c3|            _c4|         features|label|       rawPrediction|         probability|prediction|
+---+---+---+---+---------------+-----------------+-----+--------------------+--------------------+----------+
|4.5|2.3|1.3|0.3|    Iris-setosa|[4.5,2.3,1.3,0.3]|  0.0|[0.51142041330757...|[0.45890183847011...|       0.0|
|4.7|3.2|1.6|0.2|    Iris-setosa|[4.7,3.2,1.6,0.2]|  0.0|[0.82159784030878...|[0.53931700965076...|       0.0|
|4.9|2.4|3.3|1.0|Iris-versicolor|[4.9,2.4,3.3,1.0]|  1.0|[-0.1832910414978...|[0.28741978716886...|       1.0|
|4.9|3.0|1.4|0.2|    Iris-setosa|[4.9,3.0,1.4,0.2]|  0.0|[0.79189146637344...|[0.53163063661657...|       0.0|
|4.9|3.1|1.5|0.1|    Iris-setosa|[4.9,3.1,1.5,0.1]|  0.0|[0.84604516243186...|[0.54689505137992...|       0.0|
|5.0|3.0|1.6|0.2|    Iris-setosa|[5.0,3.0,1.6,0.2]|  0.0|[0.74615872130630...|[0.52022869947128...|       0.0|
|

In [155]:
# rawPrediction may vary between algorithms, but it intuitively gives a measure of confidence in each possible label (where larger = more confident).
prediction.select('rawPrediction').collect()[0][0]

DenseVector([0.5114, 0.0809, -0.1254])

In [156]:
prediction.select('probability').collect()[0][0]

DenseVector([0.4589, 0.2984, 0.2427])

In [185]:
pre_summary = prediction.summary

In [191]:
pre_summary


<bound method DataFrame.summary of DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: string, features: vector, label: double, rawPrediction: vector, probability: vector, prediction: double]>

## 2、GBDT

In [20]:
train_data.show(10)

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|5.1|3.5|1.4|0.2|Iris-setosa|
|4.9|3.0|1.4|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|5.0|3.6|1.4|0.2|Iris-setosa|
|5.4|3.9|1.7|0.4|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|5.0|3.4|1.5|0.2|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
+---+---+---+---+-----------+
only showing top 10 rows



In [21]:
train_data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]